# HFK caluculator from monodromies of fibered knots


### Modules

In [1]:
#import numpy as np
#import pandas as pd
#import ast
import random

#from multiprocessing import Pool, cpu_count
#from tqdm.notebook import tqdm

from snappy import *
#from GridPythonModule import *
#from GPM_wrapper.Grid import *
#import knot_floer_homology as hfk

## Play with SnapPy (and Twister)

In [2]:
#--- some hyperbolic fibered knots with genus up to 2
knot_data = [
 #-- alternating
 {'name': '4_1', 'md':'aB', 'alt': True},
 {'name': '6_2', 'md':'abcD', 'alt': True},
 {'name': '6_3', 'md':'abCD', 'alt': True},
 {'name': '7_7', 'md':'abCF', 'alt': True},
 {'name': '8_12', 'md':'aBcD', 'alt': True},
 #-- non-alternating
 {'name': '8_20', 'md':'abCDf', 'alt': False},
 {'name': '8_21', 'md':'abcDF', 'alt': False},
 {'name': '9_42', 'md':'abcdF', 'alt': False},
 {'name': '9_44', 'md':'abCDF', 'alt': False},
 ]

In [3]:
for k in knot_data:
    print(f"{k=}")

    #--- create the exterior by name
    M1 = Manifold(k['name'])
    print(f"{M1.volume()=}") #, {M1.DT_code()=}")

    #--- create the exterior by monodromy
    S = twister.Surface('S_2_1')
    M2 = S.bundle(k['md'], optimize=True)
    try:
        L = M2.exterior_to_link()
        pd = L.PD_code()
        print(f"{M2.volume()=}, {pd=}")
        hfk_dic = hfk.pd_to_hfk(pd)
        print(f"{hfk_dic['ranks']=}")
    except:
        print("Error: exterior_to_link failed.")
        try:
            print(f"{M2.is_isometric_to(M1)=}")
        except:
            print('Can not determine')
    print(f"---")

k={'name': '4_1', 'md': 'aB', 'alt': True}
M1.volume()=2.02988321282
Error: exterior_to_link failed.
M2.is_isometric_to(M1)=False
---
k={'name': '6_2', 'md': 'abcD', 'alt': True}
M1.volume()=4.4008325161
M2.volume()=4.4008325161, pd=[(3, 10, 4, 11), (7, 3, 8, 2), (9, 6, 10, 7), (11, 4, 0, 5), (1, 9, 2, 8), (5, 0, 6, 1)]
Error: exterior_to_link failed.
M2.is_isometric_to(M1)=True
---
k={'name': '6_3', 'md': 'abCD', 'alt': True}
M1.volume()=5.69302109128
M2.volume()=5.6930210913, pd=[(5, 9, 6, 8), (11, 7, 0, 6), (9, 2, 10, 3), (3, 10, 4, 11), (1, 4, 2, 5), (7, 1, 8, 0)]
Error: exterior_to_link failed.
M2.is_isometric_to(M1)=True
---
k={'name': '7_7', 'md': 'abCF', 'alt': True}
M1.volume()=7.6433751724
Error: exterior_to_link failed.
M2.is_isometric_to(M1)=True
---
k={'name': '8_12', 'md': 'aBcD', 'alt': True}
M1.volume()=8.9358569275
M2.volume()=8.9358569275, pd=[(7, 4, 8, 5), (9, 15, 10, 14), (3, 8, 4, 9), (11, 2, 12, 3), (15, 7, 0, 6), (1, 12, 2, 13), (13, 11, 14, 10), (5, 1, 6, 0)]
Er

In [4]:
%%timeit
S = twister.Surface('S_2_1')
M3 = S.bundle(monodromy='a*b*C*D*F') #, optimize=True) #, return_type='Triangulation')
L = M3.exterior_to_link(verbose=False) #, simplify_link=False)
print(L.DT_code())
#print(f"{M3.volume()=}, {M3.DT_code()=}")

[(16, -8, 18, -12, -14, -4, -2, -10, 6)]


KeyboardInterrupt: 

5.65 s ± 3.47

## Searching fibered genus 2 knots with negative lambda

In [2]:
# prompt: create a code generating a random word of length less than 10 with letters of {a,A,b,B,c,C,d,D,e,E,f,F}
def generate_random_word(st:int=2, mg:int=10):
    """
    Generates a random word of length less than 10 with letters from the set
    {a,A,b,B,c,C,d,D,e,E,f,F}.
    """
    letters = ['a', 'A', 'b', 'B', 'c', 'C', 'd', 'D', 'e', 'E', 'f', 'F']
    offsets = [letters[2*i+1]+letters[2*i] for i in range(6)]
    word_length = random.randint(st, st+mg)
    flag = False
    while not flag:  
        word = ''.join(random.choice(letters) for _ in range(word_length))
        stls = [word[i-1]+word[i] for i in range(len(word))]
        flag = all((stl not in offsets) and (stl[::-1] not in offsets) for stl in stls)
    return word

## Example usage
random_word = generate_random_word(10,0)
print(random_word)

FBeFdFbDCF


In [3]:
S = twister.Surface('S_2_1')
trial_limit, word_len_mgn = 5000, (10,0)

counts = {'non_S3': 0, 'nc_cn': 0,'alt': 0, 'pos_lmd': 0, 'neg_lmd': 0}
#results = []
while (counts['non_S3'] < trial_limit): # and (len(results) < 5):
    print(f"\r{counts}", end="")
    md = generate_random_word(*word_len_mgn) #; print(md)
    M3 = S.bundle(monodromy=md)
    try:
        L = M3.exterior_to_link(verbose=False)
    except Exception as e:
        counts['non_S3'] += 1
#        print(f"Error: {e}, ({count=})")
        continue
    #---
    cn = len(L.DT_code()[0])
    if cn < 17: # or cn > 26:
        counts['nc_cn'] += 1
#        print(f"\r# of crossings is {cn}")
        continue
    if L.is_alternating():
        counts['alt'] += 1
#        print(f"\r{dt} is alternating.")
        continue
    #---
    hfk_dic = L.knot_floer_homology() #hfk.pd_to_hfk(pd)
    ranks = hfk_dic['ranks']
    lmd = list(ranks.keys())[-1][1]
    if lmd < 0 or lmd > 4:
        counts['neg_lmd'] += 1
        print(f"\n\r{md, cn} \n {hfk_dic['ranks']=}")
    else:
        counts['pos_lmd'] += 1
#        print(f"{lmd=}")
#---
print(f"\n=== Finish ===")

{'non_S3': 11, 'nc_cn': 1, 'alt': 0, 'pos_lmd': 0, 'neg_lmd': 0}

KeyboardInterrupt: 

In [4]:
S = twister.Surface('S_2_1')
trial_limit, word_len_mgn = 5000, (2,3)

counts = {'non_S3': 0, 'nc_cn': 0,'alt': 0, 'pos_lmd': 0, 'neg_lmd': 0}
#results = []
while (counts['non_S3'] < trial_limit): # and (len(results) < 5):
    print(f"\r{counts}", end="")
    md = 'abCDf' + generate_random_word(*word_len_mgn) #; print(md)
    M3 = S.bundle(monodromy=md)
    try:
        L = M3.exterior_to_link(verbose=False)
    except Exception as e:
        counts['non_S3'] += 1
#        print(f"Error: {e}, ({count=})")
        continue
    #---
    cn = len(L.DT_code()[0])
    if cn < 17: # or cn > 26:
        counts['nc_cn'] += 1
#        print(f"\r# of crossings is {cn}")
        continue
    if L.is_alternating():
        counts['alt'] += 1
#        print(f"\r{dt} is alternating.")
        continue
    #---
    hfk_dic = L.knot_floer_homology() #hfk.pd_to_hfk(pd)
    ranks = hfk_dic['ranks']
    lmd = list(ranks.keys())[-1][1]
    if lmd < 0 or lmd > 4:
        counts['neg_lmd'] += 1
        print(f"\n\r{md, cn} \n {hfk_dic['ranks']=}")
    else:
        counts['pos_lmd'] += 1
#---
print(f"\n=== Finish ===")

{'non_S3': 592, 'nc_cn': 95, 'alt': 0, 'pos_lmd': 4, 'neg_lmd': 0}
('abCDfAbfCf', 31) 
 hfk_dic['ranks']={(-2, -5): 1, (-1, -4): 2, (-1, -1): 2, (0, -3): 2, (0, 0): 5, (1, -2): 2, (1, 1): 2, (2, -1): 1}
{'non_S3': 4999, 'nc_cn': 696, 'alt': 0, 'pos_lmd': 26, 'neg_lmd': 1}
=== Finish ===
